# Modélisation 

In [19]:
# Functions imports

import tqdm as notebook_tqdm

import numpy as np
import importlib
import functions
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV

from skopt import BayesSearchCV
from skopt.space import Real, Integer

import xgboost as xgb

importlib.reload(functions)
from functions import *

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 10)

In [20]:
x = pd.read_csv("../data/processed/x_processed.csv")  # 10605 rows × 10 columns
x_challenge = pd.read_csv("../data/processed/x_challenge_processed.csv")  # 4942 rows × 10 columns
x_challenge_unprocessed = pd.read_csv("../data/raw/X_test_GgyECq8.csv")
y = pd.read_csv("../data/raw/y_train_jJtXgMX.csv")  # 10605 rows × 2 columns

In [21]:
y.head(5)

,DELIVERY_START,spot_id_delta
0,2022-01-01 02:00:00+01:00,-36.874770
1,2022-01-01 03:00:00+01:00,-12.643588
2,2022-01-01 04:00:00+01:00,-1.950193
3,2022-01-01 05:00:00+01:00,1.938272
4,2022-01-01 06:00:00+01:00,0.199907


Préparation des données 

In [22]:
x_train, x_test, y_train, y_test = train_test_split(x.drop(columns=['DELIVERY_START']), y, test_size=.2)

# y_bin = pd.DataFrame({
#     'spot_id_delta' : (y['spot_id_delta'] > 0).astype(int)
# })

# y_abs = pd.DataFrame({
#     'spot_id_delta' : abs(y['spot_id_delta'])
# })

# y_bin.head(10) # Quand c'est négatif c'est 0 et quand c'est positif c'est 1 (Signe à prévoir en classfication)
# y_abs.head(10) # La valeur absolue à prévoir en régression

In [23]:
# Normalisation (pas besoin pour les arbres de décisions)

# scaler = StandardScaler()
# x = scaler.fit_transform(x)

# x = pd.DataFrame(x) # scaler renvoie des numpy.darray

In [24]:
forest = RandomForestRegressor(
    n_estimators=1200,
    min_samples_split=2, 
    min_samples_leaf=1,
    max_features='log2',
    max_depth=50,
    bootstrap=False,
    random_state=42
)

boost = xgb.XGBRegressor(
    # subsample=0.9741518731659111,
    # objective='reg:squarederror',
    # n_estimators=999,
    # min_child_weight=1,
    # max_depth=10,
    # learning_rate=0.04114063220471387,
    # reg_lambda=0.9944633236159792,
    # gamma=0.10997709421192929,
    # colsample_bytree=0.7814388855591385,
    # reg_alpha=0.6785828656004257
)



# forest_bin = RandomForestClassifier( # La RSCV n'a pas amélioré les scores, peut-être la refaire
#     n_estimators=200,
#     min_samples_split=10, 
#     min_samples_leaf=4,
#     max_features='log2',
#     max_depth=50,
#     bootstrap=False,
#     random_state=42
# )

Cross validation pour le modèle de regression

In [25]:
# cross_validation(forest, x_train, y_train['spot_id_delta'], 5, 'neg_mean_squared_error')

In [26]:
cross_validation(boost, x_train, y_train['spot_id_delta'], 5, 'neg_mean_squared_error')

##### CROSS-VALIDATION SCORES ##### 

 neg_mean_squared_error  scores from cross-validation  [ -357.94306819 -1253.22091497 -1575.47027115  -382.05440589
 -1669.98686854] 
 neg_mean_squared_error  mean from cross-validation  -1047.7351057467881 
 Standard deviation  570.4156253346758 

 ###################################


Cross validation pour le modèle de classification

In [27]:
# cross_validation(forest_bin, x, y_bin['spot_id_delta'], 5, 'accuracy')

In [28]:
# En gros c'est pas mal j'ai juste 9 en std donc je vaisfaire la RSCV pour améliorer ça surtout

# Optimisation

In [29]:
# On commence par une Randomized Search, on passera à un HyperOpt plus tard s'il faut car pour l'instant
# j'ai des problèmes avec l'importation d'un package

# Paramètres pour la RandomizedSearchCV

# param_dist_forest = {
#     'n_estimators': [int(x) for x in np.linspace(start=100, stop=1200, num=12)],
#     'max_features': [None, 'sqrt', 'log2'],
#     'max_depth': [int(x) for x in np.linspace(10, 110, num=11)] + [None],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'bootstrap': [True, False]
# }

# forest_random_search = RandomizedSearchCV (
#     estimator=forest,
#     param_distributions=param_dist_forest,
#     n_iter=100,
#     cv=3,
#     verbose=2,
#     n_jobs=-1
# )

# forest_random_search.fit(x_train, y_train['spot_id_delta'])

# print(f"Meilleurs paramètres : {forest_random_search.best_params_}")

# Meilleurs paramètres pour le regressor : {'n_estimators': 400, 'min_samples_split': 2,
#  'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 50, 'bootstrap': False}

In [30]:
# # Je fais une bayes search parce que c'est ce qui a marché pour Arthur sur le foot

# param_grid = {
#     'n_estimators': Integer(100, 1000),
#     'max_depth': Integer(3, 10),
#     'learning_rate': Real(0.01, 0.3, prior='log-uniform'),
#     'subsample': Real(0.6, 1.0),
#     'colsample_bytree': Real(0.6, 1.0),
#     'min_child_weight': Integer(1, 10),
#     'gamma': Real(0.0, 0.5),
#     'reg_alpha': Real(0.0, 1.0),
#     'reg_lambda': Real(0.0, 1.0)
# }

# # Configuration de la recherche bayésienne
# bayes_search = BayesSearchCV(
#     estimator=boost,
#     search_spaces=param_grid,
#     n_iter=50,
#     scoring='neg_mean_squared_error',
#     cv=3,
#     n_jobs=-1,
#     verbose=2,
#     random_state=42
# )

# bayes_search.fit(x_train, y_train['spot_id_delta'])

# print("Meilleurs hyperparamètres :", bayes_search.best_params_)

Prédiction

In [31]:
boost.fit(x_train, y_train['spot_id_delta'])
# forest_bin.fit(x, y_bin['spot_id_delta'])

# metrics(forest, x_test, y_test, 'Métriques pour la première RandomForest', 'regressor')

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7814388855591385, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0.10997709421192929,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.04114063220471387,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=999, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [32]:
# Je fais une weighted accuracy pour pouvoir évaluer moi-même les performances du modèle 
# au lieu de devoir soumettre sur le site de l'ENS.

def weighted_accuracy(y_true, y_pred, weights):

    # Déterminer les prédictions correctes en termes de signe
    correct_predictions = np.sign(y_true) == np.sign(y_pred)
    
    # Calculer la weighted accuracy
    weighted_acc = np.sum(correct_predictions * weights) / np.sum(weights)
    return weighted_acc

In [33]:
weights = np.abs(y_test['spot_id_delta'])

y_pred = boost.predict(x_test)

# Évaluation
mse = mean_squared_error(y_test['spot_id_delta'], y_pred)
weighted_accc = weighted_accuracy(y_test['spot_id_delta'], y_pred, weights)

print(f"Mean squarred error : {mse:.2f}") # Les prédictions sont proches des valeurs réelles ou non
print(f"Weighted accuracy : {weighted_accc:.2f}") # Qualité à prédire les signes sur les valeurs importantes

# Vis à vis de cette sortie, forest est bien meilleure que boost

Mean squarred error : 306.04
Weighted accuracy : 0.93


In [34]:
y_challenge = pd.DataFrame({ # Faut prendre 'x_challenge_unprocessed' parce que sinon y'a le décalage horaire
    'DELIVERY_START' : x_challenge_unprocessed['DELIVERY_START']  
    })

y_challenge['spot_id_delta'] = boost.predict(x_challenge.drop(columns=['DELIVERY_START']))
# y_pred['spot_id_delta_bin'] = forest_bin.predict(x_challenge)


y_challenge.head(5)

,DELIVERY_START,spot_id_delta
0,2023-04-02 00:00:00+02:00,6.229511
1,2023-04-02 01:00:00+02:00,3.208276
2,2023-04-02 02:00:00+02:00,-1.771261
3,2023-04-02 03:00:00+02:00,-0.497673
4,2023-04-02 04:00:00+02:00,0.774299


In [35]:
# y_pred['spot_id_delta_bin'] = y_pred['spot_id_delta_bin'].apply(lambda x: 1 if x==1 else -1)

# y_pred['spot_id_delta'] = y_pred['spot_id_delta_abs'] * y_pred['spot_id_delta_bin']
# y_pred = y_pred.drop(columns=['spot_id_delta_abs', 'spot_id_delta_bin'])

# y_pred.head(5)

In [36]:
y_challenge.to_csv("../data/predicted/y_challenge_10.csv", index=False)